In [134]:
import pandas as pd
import numpy as np
import datetime

#### Data Cleaning


In [200]:
device_net = pd.read_csv('/content/device_network_data.csv')
device_net

,Epoch Time,Value
0,1704042000000,1
1,1704042026000,1
2,1704042035000,1
3,1704042037000,1
4,1704042059000,1
...,...,...
7214,1704128340000,1
7215,1704128361000,1
7216,1704128378000,Null
7217,1704128391000,0


Check Data Type and Missing Data

In [201]:
device_net.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Epoch Time  7219 non-null   int64 
 1   Value       7216 non-null   object
dtypes: int64(1), object(1)
memory usage: 112.9+ KB


In [209]:
# checking unique Value class
device_net['Value'].unique()

array(['1', '0', 'Null', nan], dtype=object)

In [210]:
device_net['Value'].value_counts().to_dict()

{'1': 4941, '0': 2268, 'Null': 7}

In [211]:
# convert Null to nan value so then we can do data imputation with the same method
device_net['Value'] = device_net['Value'].replace('Null', np.nan)

In [212]:
# imputate Null and Nan value with LOCF (Last Observation Carried Forward) method
device_net_imputed = device_net.ffill()

In [214]:
device_net_imputed['Value'].value_counts().to_dict()

{'1': 4945, '0': 2274}

In [215]:
# convert Value type into numeric
device_net_imputed['Value'] = device_net_imputed['Value'].astype('int')

In [216]:
device_net_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Epoch Time  7219 non-null   int64
 1   Value       7219 non-null   int64
dtypes: int64(2)
memory usage: 112.9 KB


And then we wanted to convert the epoch time or unix time* into datetime for more readable format and simplified calculation then

*Unix time is the number of seconds that have elapsed since the Unix epoch, 00:00:00 UTC (Coordinated Universal Time) on January 1, 1970

In [217]:
# Convert time
device_net_imputed['DateTime'] = pd.to_datetime(device_net_imputed['Epoch Time'], unit='ms')

In [218]:
device_net_imputed

,Epoch Time,Value,DateTime
0,1704042000000,1,2023-12-31 17:00:00
1,1704042026000,1,2023-12-31 17:00:26
2,1704042035000,1,2023-12-31 17:00:35
3,1704042037000,1,2023-12-31 17:00:37
4,1704042059000,1,2023-12-31 17:00:59
...,...,...,...
7214,1704128340000,1,2024-01-01 16:59:00
7215,1704128361000,1,2024-01-01 16:59:21
7216,1704128378000,1,2024-01-01 16:59:38
7217,1704128391000,0,2024-01-01 16:59:51


#### Calculate Downtime total Duration

In [225]:
# using example data to make function

data_test_1 = [
    (1704042000000, 0),
    (1704042026000, 1),
    (1704042035000, 1),
    (1704042037000, 0),
    (1704042059000, 1),
    (1704042060000, 1)
]

data_test_1 = pd.DataFrame(data_test_1, columns=['datetime', 'value'])
data_test_1['datetime'] = pd.to_datetime(data_test_1['datetime'], unit='ms')
print(data_test_1)

data_test_2 = [
    (1704042000000, 1),
    (1704042026000, 1),
    (1704042035000, 1),
    (1704042037000, 1),
    (1704042059000, 1),]

data_test_2 = pd.DataFrame(data_test_1, columns=['datetime', 'value'])
data_test_2['datetime'] = pd.to_datetime(data_test_2['datetime'], unit='ms')
print(data_test_2)

             datetime  value
0 2023-12-31 17:00:00      0
1 2023-12-31 17:00:26      1
2 2023-12-31 17:00:35      1
3 2023-12-31 17:00:37      0
4 2023-12-31 17:00:59      1
5 2023-12-31 17:01:00      1
             datetime  value
0 2023-12-31 17:00:00      0
1 2023-12-31 17:00:26      1
2 2023-12-31 17:00:35      1
3 2023-12-31 17:00:37      0
4 2023-12-31 17:00:59      1
5 2023-12-31 17:01:00      1


As from the example data, the data_test_1 expected to have 12 second for total duration of downtime and data_test_2 expected to have 59 second total downtime duration.

Each row in the dataset contains a timestamp and a status value. The status can be either 'down' (represented by 1) or 'up' (represented by 0).

Then, it can be assume that:
- For each segment where the system status is 'down' (1) then
look for the next occurrence where the status changes to 'up' (0).
And calculate difference between the timestamp when the system was 'down' and the timestamp when it transitions to 'up' (datetime(value=0) - datetime(value=1))
- If the system remains 'down' (1) until the end of the dataset without transitioning to 'up' (0), calculate the total downtime as the difference between the timestamp of the first 'down' entry and the last 'down' entry.

Accumulate these durations to get the total downtime.



In [ ]:
'''
Or the calculation will be like this for data_test_1:

Segment 1:

Start: 2023-12-31 17:00:26 (value is 1)
End: 2023-12-31 17:00:37 (value transitions to 0)
Duration: 17:00:37 - 17:00:26 = 11 seconds

Segment 2:

Start: 2023-12-31 17:00:59 (value is 1)
End: 2023-12-31 17:01:00 (value transitions to 0)
Duration: 17:01:00 - 17:00:59 = 1 second

Total = Segment 1 + Segment 2 = 12 second

and for data_test_2:

Since the dataset represents continuous 'down' values, the downtime should be calculated as the difference between the first
timestamp when the system is 'down' and the last timestamp of the 'down' period.

Duration = 17:00:59 - 17:00:00 = 59 second
'''

With those assumtions we can make a function to calculate total downtime

In [233]:
def calculate_downtime(df):
    '''
    1. mask all 'down' value or the value is 1, so if the value is 1 then it masked True and vice versa
    2. next_time contains the timestamp of the next row for each current row
    3. calculate difference between next_time and the current timestamp for each row and convert to second
    4. Accumulate total downtime with sum it all
    '''
    mask = df.Value == 1
    next_time = df.DateTime.shift(-1)
    downtime = (next_time - df.DateTime).dt.total_seconds()
    total_down = downtime[mask].sum()

    return total_down

In [234]:
calculate_downtime(device_net_imputed)

58310.0

In [238]:
import time

seconds = calculate_downtime(device_net_imputed)
convert = time.strftime("%H:%M:%S", time.gmtime(seconds))
convert

'16:11:50'

In [239]:
print("End: ", device_net_imputed['DateTime'].max())
print("Start: ", device_net_imputed['DateTime'].min())

End:  2024-01-01 17:00:00
Start:  2023-12-31 17:00:00


Therefore, total downtime of the device from 2023-12-31 17:00:00 to 2024-01-01 17:00:00 is 58310 seconds or 16 hours 11 minutes 50 seconds.